# Deep Learning for Business Applications course

## TOPIC 7: Face Detection

### 1. Library installation

Documentation for use of OpenCV with Python API [see here](https://docs.opencv.org/).

In [ ]:
!pip3 install opencv-python

In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

### 2. Haarcascades approach

In [ ]:
# load the required trained XML classifiers
# https://github.com/Itseez/opencv/blob/master/
# data/haarcascades/haarcascade_frontalface_default.xml
# Trained XML classifiers describes some features of some
# object we want to detect a cascade function is trained
# from a lot of positive(faces) and negative(non-faces)
# images.
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# https://github.com/Itseez/opencv/blob/master
# /data/haarcascades/haarcascade_eye.xml
# Trained XML file for detecting eyes
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 

In [ ]:
face_cascade

In [ ]:
file_path = 'imgs/faces1.jpg'
img = cv2.imread(file_path)
assert img is not None, 'file could not be read, check if file exists'

In [ ]:
# RGBimage = cv2.cvtColor(BGRimage, cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(16, 8))
plt.imshow(img)
plt.show()

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# Detects faces of different sizes in the input image
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

In [ ]:
for i, (x,y,w,h) in enumerate(faces):
    # To draw a rectangle in a face 
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2) 
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    cv2.putText(
        img,
        f'face {i}',
        (x, y),
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (255, 255, 0),
        3
    )

    # Detects eyes of different sizes in the input image
    eyes = eye_cascade.detectMultiScale(roi_gray) 

    # To draw a rectangle in eyes
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,127,255),2)

In [ ]:
# Display an image in a window
plt.figure(figsize=(16, 8))
plt.imshow(img)
plt.show()

### 3. CNN approach

In [ ]:
!pip install transformers

In [ ]:
import torch
import numpy as np
from PIL import Image
from transformers import DetrImageProcessor, DetrForObjectDetection

In [ ]:
img_path = f'imgs/faces1.jpg'
img = Image.open(img_path)
img_ = np.array(img)
plt.figure(figsize=(16, 6))
plt.imshow(img)
plt.show()

In [ ]:
# model and image processor
model_name = 'diffusionai/detr-face-detection'
processor = DetrImageProcessor.from_pretrained(model_name)
model = DetrForObjectDetection.from_pretrained(model_name)

# inference for detection
inputs = processor(images=img, return_tensors='pt')
outputs = model(**inputs)

In [ ]:
# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
th = .9
target_sizes = torch.tensor([img.size[::-1]])
results = processor.post_process_object_detection(
    outputs,
    target_sizes=target_sizes,
    threshold=th
)[0]

# results and bbox drawing
for i, (score, box) in enumerate(zip(results['scores'], results['boxes'])):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f'detected face {i} with confidence',
            f'{round(score.item(), 2)} at location {box}'
    )

    top_left = (int(box[0]), int(box[1]))
    bottom_right = (int(box[2]), int(box[3]))
    cv2.rectangle(img_, top_left, bottom_right, (0, 255, 0), 3)
    cv2.putText(
        img_,
        f'face {i}',
        top_left,
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (0, 255, 0),
        3
    )

In [ ]:
plt.figure(figsize=(16, 6))
plt.imshow(img_)
plt.show()